<a href="https://colab.research.google.com/github/Bhaskar-Majumdar/Fake-News-LSTM/blob/main/xgboost_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h4>Fake News Classification. (Task3a) </h4>
    <p>Fake news classifier model with 4 output classess:</p>
    <ol>
    <li>FALSE</li>
    <li>TRUE</li>
    <li>partially false</li>
    <li>other</li>
    </ol>
<h5>This machine learning model is a XGBclassifier model that usually works very well with text data</h5>

Importing processed final training data and test data as pandas dataframe:

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('final_stemmed_data.csv')
test = pd.read_csv('Task3a_testing.csv')

In [ ]:
# Checking For null values
print(len(train))
print(train.isnull().sum())

59494
text          633
our rating      0
dtype: int64


In [ ]:
# dropping all the null values
train=train.dropna(axis=0)
train.reset_index(inplace=True)
train = train.drop('index', axis=1)

In [ ]:
# After droping null values
print(len(train))
print(train.isnull().sum())

58861
text          0
our rating    0
dtype: int64


<h4>Text processing functions.</h4>
<h6> This functions will convert our Task3a_testing data to our Classifier feedable vector data so that we can predict of from the test data. </h6>

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# text corpus to count_vectorized data converter function
def count_vectorization(corpus, max_features = 5000, ngram_range = (1,3)):
    
    cv = CountVectorizer(max_features=max_features,ngram_range=ngram_range)
    X = cv.fit_transform(corpus).toarray()
    
    return (X,cv)

In [ ]:
# stemming and stop words removing function
def text_process(df):
    ps = PorterStemmer()
    corpus = []
    for i in range(0, len(df)):
        print(i)
        review = re.sub('[^a-zA-Z]', ' ', str(df['input'][i]))
        review = review.lower()
        review = review.split()

        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
        
    
    return corpus

In [ ]:
# splitting training data into data and label as  X_data and y_data 
X_data=train.drop(['our rating'],axis=1)
y_data=train['our rating']

In [ ]:
X_data.head()

,text
0,last week rep louie gohmert told chri salcedo ...
1,washington reuter head conserv republican fact...
2,donald trump wish american happi new year leav...
3,distract drive caus death canada impair drive ...
4,whatev drama play republican meet cleveland ne...


In [ ]:
y_data.unique()

array(['FALSE', 'TRUE', 'partially false', 'other'], dtype=object)

In [ ]:
X_data, cv = count_vectorization(X_data['text'])

In [ ]:
print(X_data.shape)
print(y_data.shape)

(58861, 5000)
(58861,)


<h6>Splitting total trainig data into trainig and validation data with 80:20 ratio.</h6>

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=123)

<h3>Defining our ML classifier</h3>
<h3>This classifier model is XGBClassifer from xgboost library</h3>

In [ ]:
from xgboost import XGBClassifier

In [ ]:
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

C:\Users\user\anaconda3\envs\lab_env\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:04:53] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=6, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
Y_pred = classifier.predict(X_test)

Viewing classification report:

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
print(classification_report(y_test, Y_pred)) 

                 precision    recall  f1-score   support

          FALSE       0.89      0.98      0.94      6596
           TRUE       0.99      0.99      0.99      4326
          other       0.49      0.11      0.19       819
partially false       0.57      0.12      0.21        32

       accuracy                           0.92     11773
      macro avg       0.74      0.55      0.58     11773
   weighted avg       0.90      0.92      0.90     11773



In [ ]:
print('Model Accuracy: ',accuracy_score(y_test,Y_pred))

Model Accuracy:  0.9227894334494181


<h3>Prediction of the test set</h3>
<h4>And exporting the result as a tsv file</h4>

In [ ]:
test.head()

,public_id,text,title,our rating
0,81a67c96,Former state House Majority Leader Adam Hasner...,- The Washington Post,NaN
1,6e5ec6fb,"Editor’s note: Subsequent to this article, VTD...",Rubio Comments on Iran Nuclear Deal,NaN
2,d9cd4895,The hypocritical Lib Dems want to ignore the r...,Climate Alarmists Caught Manipulating Temperat...,NaN
3,4a1a9b9f,Urgent action to boost the number of children ...,Who are the arsonists setting rural fires in W...,NaN
4,6d16fa40,By Ken Allen AFSCME Council 75 represents the ...,"Diabetes prescriptions now cost NHS £1bn, figu...",NaN


In [ ]:
# merging title and the text into a single data form as input
test['input'] = test['title']+' '+test['text']

In [ ]:
# stemming and removing stop words with the help of our predefined text_process() function
corpus = text_process(test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
#converting our cleaned text_corpus into count_vectorize data with predefined count_vectorization() function
test_data, cv = count_vectorization(corpus)

In [ ]:
# prediction of the rating(FALSE,TRUE,partially false,other) of Task3a_testing.csv dataset with our classifier.
predicted_rating = classifier.predict(test_data)

In [ ]:
predicted_rating

array(['TRUE', 'FALSE', 'FALSE', 'other', 'FALSE', 'TRUE', 'FALSE',
       'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE',
       'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE',
       'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE',
       'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE',
       'FALSE', 'FALSE', 'TRUE', 'FALSE', 'other', 'FALSE', 'FALSE',
       'FALSE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'FALSE',
       'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE',
       'TRUE', 'FALSE', 'TRUE', 'FALSE', 'other', 'FALSE', 'FALSE',
       'FALSE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE',
       'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE',
       'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE',
       'TRUE', 'other', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE',
       'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'other', 'FALSE',
       'other', 'FALSE', 'FALSE', 'FA

<h5>Exporting the result as tsv with 'public_id' and 'predicted_rating' column.</h5>

In [ ]:
task3a_df = pd.DataFrame()

In [ ]:
task3a_df['public_id'] = test['public_id']
task3a_df['predicted_rating'] = predicted_rating

In [ ]:
task3a_df.head()

,public_id,predicted_rating
0,81a67c96,TRUE
1,6e5ec6fb,FALSE
2,d9cd4895,FALSE
3,4a1a9b9f,other
4,6d16fa40,FALSE


In [ ]:
task3a_df.to_csv('result/task3a.tsv', sep='\t', index=False)

<h6> Saving our trained classifer model.</h6>

In [ ]:
#import pickle

In [ ]:
"""# save model to disk
filename = 'xgbc_model.sav'
pickle.dump(classifier, open(filename, 'wb'))"""

In [ ]:
"""# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)"""

0.9227894334494181
